<a href="https://colab.research.google.com/github/shishc9/blogSystem-backend/blob/master/machinelearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import sys
import random

trainpath = '/content/drive/MyDrive/Machine/train'
validationpath = '/content/drive/MyDrive/Machine/validation'
num_sample = 30    #定义随机采样数量 
if not os.path.exists(validationpath):
    os.mkdir(validationpath)
else:
    print('error path already exists!!')

folderlist = os.listdir(trainpath)
print('folder list : {}'.format(folderlist))
for dirpath in folderlist:
    if not os.path.exists(os.path.join(validationpath, dirpath)):
        os.mkdir(os.path.join(validationpath, dirpath))
    loadpath = os.path.join(trainpath,dirpath)
    datalist = os.listdir(loadpath)
    samplelist = random.sample(datalist, num_sample)
    for filename in samplelist:
        os.renames(os.path.join(loadpath, filename), os.path.join(validationpath, dirpath, filename))


print('Sampling completed')

error path already exists!!
folder list : ['sea', 'ship']
Sampling completed


In [6]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import torchvision.datasets as dset
import matplotlib.pyplot as plt
import os
import copy

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")


# device check
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('runing device : {}'.format(device))


# Training settings
batch_size = 32


train_set = dset.ImageFolder(root='/content/drive/MyDrive/Machine/train',transform = transforms.Compose([
                                transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor()
                               ]))

validation_set = dset.ImageFolder(root='/content/drive/MyDrive/Machine/validation',transform = transforms.Compose([
                                transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()]))


# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_set,
                                          batch_size=batch_size,
                                          shuffle=False)


# Load the pretrained model from pytorch
# vgg16 = models.vgg16(pretrained=True)
### 当pretrained=True时，运行时会自动下载vgg16-397923af.pth这个预训练模型文件到你的电脑，下载完成后只需要将下载的模型文件按照路径加载进来就可以使用了，需要注意，如果使用预加载训练模型时，需要将trianed=False
vgg16 = models.vgg16(pretrained=False)
vgg16.load_state_dict(torch.load("/root/.cache/torch/hub/checkpoints/vgg16-397923af.pth"))
print(vgg16.classifier[6].out_features) # 1000




# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False


# Newly created modules have require_grad=True by default
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
# 我这里是2类目标，所以应该是2
features.extend([nn.Linear(num_features, 2)]) # Add our layer with 2 outputs
features.extend([nn.LogSoftmax(1)])
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier



# If you want to train the model for more than 2 epochs, set this to True after the first run
resume_training = False

if resume_training:
    print("Loading pretrained model..")
    vgg16.load_state_dict(torch.load('./vgg16-397923af.pth'))
    print("Loaded!")

vgg16.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

def train(vgg, optimizer, epoch, num_epochs=10):
    vgg.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = vgg(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            #     num_epochs, batch_idx * len(data), len(train_loader.dataset),
            #     100. * batch_idx / len(train_loader), loss.data[0]))
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch*num_epochs+batch_idx, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.tolist()))


def test(vgg):
    vgg.eval()
    test_loss = 0
    correct = 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = vgg(data)
        # sum up batch loss
        # test_loss += F.nll_loss(output, target, size_average=False).data[0]
        test_loss += F.nll_loss(output, target, size_average=False).tolist()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(validation_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(validation_loader.dataset),
        100. * correct / len(validation_loader.dataset)))
    return correct




for epoch in range(5):
    train(vgg16, optimizer_ft, epoch)
    torch.save(vgg16, 'best#111.pb')
test(vgg16)
torch.save(vgg16, 'best#111.pb')

Using CUDA
runing device : cuda:0


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


FileNotFoundError: ignored

In [8]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import torchvision.datasets as dset
import matplotlib.pyplot as plt
import os
import copy

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")


# device check
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('runing device : {}'.format(device))


# Training settings
batch_size = 32


train_set = dset.ImageFolder(root='/content/drive/MyDrive/Machine/train',transform = transforms.Compose([
                                transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor()
                               ]))

validation_set = dset.ImageFolder(root='/content/drive/MyDrive/Machine/validation',transform = transforms.Compose([
                                transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()]))


# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_set,
                                          batch_size=batch_size,
                                          shuffle=False)


# Load the pretrained model from pytorch
# vgg16 = models.vgg16(pretrained=True)
### 当pretrained=True时，运行时会自动下载vgg16-397923af.pth这个预训练模型文件到你的电脑，下载完成后只需要将下载的模型文件按照路径加载进来就可以使用了，需要注意，如果使用预加载训练模型时，需要将trianed=False
vgg16 = models.vgg16(pretrained=False)
vgg16.load_state_dict(torch.load("/root/.cache/torch/hub/checkpoints/vgg16-397923af.pth"))
print(vgg16.classifier[6].out_features) # 1000




# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False


# Newly created modules have require_grad=True by default
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
# 我这里是2类目标，所以应该是2
features.extend([nn.Linear(num_features, 2)]) # Add our layer with 2 outputs
features.extend([nn.LogSoftmax(1)])
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier



# If you want to train the model for more than 2 epochs, set this to True after the first run
resume_training = False

if resume_training:
    print("Loading pretrained model..")
    vgg16.load_state_dict(torch.load('./vgg16-397923af.pth'))
    print("Loaded!")

vgg16.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

def train(vgg, optimizer, epoch, num_epochs=10):
    vgg.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = vgg(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            #     num_epochs, batch_idx * len(data), len(train_loader.dataset),
            #     100. * batch_idx / len(train_loader), loss.data[0]))
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch*num_epochs+batch_idx, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.tolist()))


def test(vgg):
    vgg.eval()
    test_loss = 0
    correct = 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = vgg(data)
        # sum up batch loss
        # test_loss += F.nll_loss(output, target, size_average=False).data[0]
        test_loss += F.nll_loss(output, target, size_average=False).tolist()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(validation_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(validation_loader.dataset),
        100. * correct / len(validation_loader.dataset)))
    return correct




for epoch in range(5):
    train(vgg16, optimizer_ft, epoch)
    torch.save(vgg16, 'best#111.pb')
test(vgg16)
torch.save(vgg16, 'best#111.pb')

Using CUDA
runing device : cuda:0
1000
Train Epoch: 0 [0/2580 (0%)]	Loss: 0.796237
Train Epoch: 10 [320/2580 (12%)]	Loss: 0.483265
Train Epoch: 20 [640/2580 (25%)]	Loss: 0.405360
Train Epoch: 30 [960/2580 (37%)]	Loss: 0.105630
Train Epoch: 40 [1280/2580 (49%)]	Loss: 0.206417
Train Epoch: 50 [1600/2580 (62%)]	Loss: 0.358646
Train Epoch: 60 [1920/2580 (74%)]	Loss: 0.155632
Train Epoch: 70 [2240/2580 (86%)]	Loss: 0.213767
Train Epoch: 80 [1600/2580 (99%)]	Loss: 0.044564
Train Epoch: 10 [0/2580 (0%)]	Loss: 0.226952
Train Epoch: 20 [320/2580 (12%)]	Loss: 0.251693
Train Epoch: 30 [640/2580 (25%)]	Loss: 0.370372
Train Epoch: 40 [960/2580 (37%)]	Loss: 0.404641
Train Epoch: 50 [1280/2580 (49%)]	Loss: 0.222951
Train Epoch: 60 [1600/2580 (62%)]	Loss: 0.115628
Train Epoch: 70 [1920/2580 (74%)]	Loss: 0.278385
Train Epoch: 80 [2240/2580 (86%)]	Loss: 0.131753
Train Epoch: 90 [1600/2580 (99%)]	Loss: 0.163975
Train Epoch: 20 [0/2580 (0%)]	Loss: 0.200315
Train Epoch: 30 [320/2580 (12%)]	Loss: 0.025499
T

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1000, Accuracy: 687/716 (96%)



In [12]:
from __future__ import print_function
import torch
from torchvision.transforms import RandomCrop, Resize, Compose, ToTensor
import numpy as np
import time
import os
import cv2

imagesize = (224,224,3)
labeldict = {'0' : 'sea', '1' : 'ship'}

def predict(model, imgarr, device):
    inimage = cv2.cvtColor(imgarr, cv2.COLOR_BGR2RGB)
    inimage = cv2.resize(inimage, (imagesize[0], imagesize[1]))
    inimage = np.transpose(inimage, (2, 0, 1)) / 255
    inimage = np.expand_dims(inimage, axis=0)
    inimage = torch.tensor(inimage.astype('float32'))
    inimage = inimage.to(device)
    netout = model(torch.tensor(inimage))
    label = str(netout.argmax().item())
    return labeldict[label]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('runing device : {}'.format(device))
model = torch.load('./best#111.pb')
model.to(device)
model.eval()

testdir = '/content/drive/MyDrive/Machine/test/ship'
for filename in os.listdir(testdir):
    filepath = os.path.join(testdir, filename)
    img = cv2.imread(filepath)
    #rec_c = cv2.cvtColor(rec_c, cv2.COLOR)
    # print(img.shape)
    sol = predict(model, img, device)
    print(filename +'prediction : {}'.format(sol))

runing device : cuda:0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


ship__20180705_182236_0f4d__-122.3334438500883_37.75037217054128.pngprediction : ship
ship__20171118_181532_1030__-122.34400966135502_37.75705610175418.pngprediction : ship
ship__20171118_185722_0f2d__-122.34496788529589_37.73721449905369.pngprediction : ship
ship__20171025_175648_0e26__-118.15349753221352_33.69822802602944.pngprediction : ship
ship__20171217_181637_1032__-122.33377734078427_37.728363954110335.pngprediction : ship
ship__20171118_185723_0f2d__-122.35675214597191_37.79184481316763.pngprediction : sea
ship__20171023_175305_1039__-118.11559165475667_33.65552976108269.pngprediction : ship
ship__20180205_181812_1030__-122.32540372847062_37.72823521408221.pngprediction : ship
ship__20171023_175305_1039__-118.10923630341375_33.67963996316044.pngprediction : ship
ship__20171127_181539_101e__-122.33856455895739_37.71068001749797.pngprediction : ship
ship__20180312_182016_1042__-122.33736834649797_37.753260475183126.pngprediction : ship
ship__20171129_181545_1022__-122.3427732037